In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from matplotlib import pyplot as plt
import pickle
import import_ipynb
import sys
sys.path.append('./../')
import Portfolio_value as pv
import statsmodels.api as sm

In [2]:
df =  pd.read_csv('./../data/data.csv')

In [3]:
p_val = np.load('./../result/p_val/p_val_const_delaying.npy', allow_pickle=True)
w_val = np.load('./../result/w_val/w_val_const_delaying.npy', allow_pickle=True)

p_val = p_val.reshape(1)
p_val = p_val[0]

w_val = w_val.reshape(1)
w_val = w_val[0]

In [4]:

# for upper_year in [15,20,25]:
#     for k in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
#         for withdraw_rate in [0.04,0.06,0.08,0.10]:
#             for year in range(1980,2003-upper_year):
             

In [5]:
normal_year = np.load('./../result/year/normal_year.npy', allow_pickle=True).reshape(1)[0]
bear_year = np.load('./../result/year/bear_year.npy', allow_pickle=True).reshape(1)[0]
full_year = np.load('./../result/year/full_year.npy', allow_pickle=True).reshape(1)[0]

In [6]:
upper_years = [20,25,30]#[10, 15, 20] #[15, 20, 25]#[10, 15, 20] # , 

In [7]:
normal_perform = {}
bear_perform = {}
full_perform = {}
for upper_year in upper_years:
    normal_perform[upper_year] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}
    bear_perform[upper_year] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}
    
    full_perform[upper_year] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}

In [8]:
#      0 / 5 / 10
# 8 -> 5.1 / 4.7 / 4.3
# 6 -> 3 / 2.6 / 2.1
# 4 -> 1.3 / 0.9 / 0.5

# 0

In [9]:
init_wealth = 350000
k = 0.5 # 고정

x = {
    10:[],
    15:[],
    20:[],
    25:[],
    30:[]
}
depletion = {
    10:[],
    15:[],
    20:[],
    25:[],
    30:[]
    
}
wealth = {
    10:[],
    15:[],
    20:[],
    25:[],
    30:[]
    
}

for upper_year in upper_years:
    print('upper_year:', upper_year)
    if upper_year == 20:
        withdraw_rates = [0.051, 0.03]
    elif upper_year == 25:
        withdraw_rates = [0.047, 0.026]
    elif upper_year == 30:
        withdraw_rates = [0.043, 0.021]
        
    for withdraw_rate in withdraw_rates:
        for years in full_year[upper_year]:
            for year in years:
                print('year:', year)
                if year in normal_year[upper_year][0]:
                    x[upper_year].append(0)
                elif year in bear_year[upper_year][0]:
                    x[upper_year].append(1)

                res = pv.get_metric(year, k, upper_year, withdraw_rate, df, init_wealth, p_val, w_val)
                if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                    dep_or_not = 1
                elif res[2] >= upper_year*12: # 고갈이 안되면
                    dep_or_not = 0

                depletion[upper_year].append(dep_or_not)
                wealth[upper_year].append((res[1] + res[0])*350000/100/1000)



upper_year: 20
year: 1980
year: 1981
year: 1982
year: 1983
year: 1984
year: 1985
year: 1986
year: 1987
year: 1988
year: 1989
year: 1990
year: 1991
year: 1992
year: 1993
year: 1994
year: 1995
year: 1996
year: 1997
year: 1998
year: 1999
year: 2000
year: 2001
year: 2002
year: 1980
year: 1981
year: 1982
year: 1983
year: 1984
year: 1985
year: 1986
year: 1987
year: 1988
year: 1989
year: 1990
year: 1991
year: 1992
year: 1993
year: 1994
year: 1995
year: 1996
year: 1997
year: 1998
year: 1999
year: 2000
year: 2001
year: 2002
upper_year: 25
year: 1980
year: 1981
year: 1982
year: 1983
year: 1984
year: 1985
year: 1986
year: 1987
year: 1988
year: 1989
year: 1990
year: 1991
year: 1992
year: 1993
year: 1994
year: 1995
year: 1996
year: 1997
year: 1980
year: 1981
year: 1982
year: 1983
year: 1984
year: 1985
year: 1986
year: 1987
year: 1988
year: 1989
year: 1990
year: 1991
year: 1992
year: 1993
year: 1994
year: 1995
year: 1996
year: 1997
upper_year: 30
year: 1980
year: 1981
year: 1982
year: 1983
year: 198

In [10]:
intercept = {}
beta = {}
t_intercept = {}
t_beta = {}
r2 = {}

intercept_total = {}
beta_total = {}
t_intercept_total = {}
t_beta_total = {}
r2_total = {}

for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.Logit(depletion[upper_year], x_)#sm.OLS(depletion[upper_year], x_)
    res = mod.fit()
    print(res.summary())
#     print(res.prsquared)
    intercept[upper_year] = np.round(res.params[0],3)
    beta[upper_year] = np.round(res.params[1],3)
    
    t_intercept[upper_year] = np.round(res.tvalues[0],3)
    t_beta[upper_year] = np.round(res.tvalues[1],3)
    
    r2[upper_year] = np.round(res.prsquared, 3)
    
for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.OLS(wealth[upper_year], x_)
    res = mod.fit()
    print(res.summary())
#     print(res.prsquared)

    intercept_total[upper_year] = np.round(res.params[0],3)
    beta_total[upper_year] = np.round(res.params[1],3)
    
    t_intercept_total[upper_year] = np.round(res.tvalues[0],3)
    t_beta_total[upper_year] = np.round(res.tvalues[1],3)
    
    r2_total[upper_year] = np.round(res.rsquared_adj, 3)

upper_year: 20
         Current function value: 0.415118
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                   46
Model:                          Logit   Df Residuals:                       44
Method:                           MLE   Df Model:                            1
Date:                Sun, 09 Jul 2023   Pseudo R-squ.:                  0.2072
Time:                        22:38:40   Log-Likelihood:                -19.095
converged:                      False   LL-Null:                       -24.085
Covariance Type:            nonrobust   LLR p-value:                  0.001583
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            34.1779   8.85e+06   3.86e-06      1.000   -1.73e+07    1.73e+07
const        -34.8710   8.85e+06  -3.94e-06      1

/home/slcf/anaconda3/envs/hyungjinko/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [11]:
print('Intercept & & {} & {} & & {} & {} & & {} & {}'.format(intercept[20], intercept_total[20], intercept[25], intercept_total[25], intercept[30], intercept_total[30]))
print(' & & ({}) & ({}) & & ({}) & ({}) & & ({}) & ({})'.format(t_intercept[20], t_intercept_total[20], t_intercept[25], t_intercept_total[25], t_intercept[30], t_intercept_total[30]))
print('Bearish (dummy) & & {} & {} & & {} & {} & & {} & {}'.format(beta[20], beta_total[20], beta[25], beta_total[25], beta[30], beta_total[30]))
print(' & & ({}) & ({}) & & ({}) & ({}) & & ({}) & ({})'.format(t_beta[20], t_beta_total[20], t_beta[25], t_beta_total[25], t_beta[30], t_beta_total[30]))



Intercept & & 34.178 & -145.194 & & 2.708 & -216.567 & & 2.708 & -236.534
 & & (0.0) & (-7.264) & & (2.406) & (-12.737) & & (2.236) & (-10.248)
Bearish (dummy) & & -34.871 & 467.825 & & -2.708 & 496.712 & & -2.708 & 505.569
 & & (-0.0) & (28.981) & & (-2.622) & (39.192) & & (-2.622) & (35.32)


In [12]:
print('Adj. $R^2$ & & {} & {} & & {} & {} & & {} & {}'.format(r2[20], r2_total[20], r2[25], r2_total[25], r2[30], r2_total[30]))


Adj. $R^2$ & & 0.207 & 0.535 & & 0.206 & 0.822 & & 0.24 & 0.806


# 5

In [13]:
upper_years = [15, 20, 25]#[10, 15, 20] # , 

In [14]:
init_wealth = 350000
k = 0.5 # 고정

x = {
    10:[],
    15:[],
    20:[],
    25:[]
}
depletion = {
    10:[],
    15:[],
    20:[],
    25:[]
    
}
wealth = {
    10:[],
    15:[],
    20:[],
    25:[]
    
}

for upper_year in upper_years:
    print('upper_year:', upper_year)
    if upper_year == 15:
        withdraw_rates = [0.051, 0.03]
    elif upper_year == 20:
        withdraw_rates = [0.047, 0.026]
    elif upper_year == 25:
        withdraw_rates = [0.043, 0.021]
        
    for withdraw_rate in withdraw_rates:
        for years in full_year[upper_year]:
            for year in years:
                print('year:', year)
                if year in normal_year[upper_year][0]:
                    x[upper_year].append(0)
                elif year in bear_year[upper_year][0]:
                    x[upper_year].append(1)

                res = pv.get_metric(year, k, upper_year, withdraw_rate, df, init_wealth, p_val, w_val)
                if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                    dep_or_not = 1
                elif res[2] >= upper_year*12: # 고갈이 안되면
                    dep_or_not = 0

                depletion[upper_year].append(dep_or_not)
                wealth[upper_year].append((res[1] + res[0])*350000/100/1000)



upper_year: 15
year: 1980
year: 1981
year: 1982
year: 1983
year: 1984
year: 1985
year: 1986
year: 1987
year: 1988
year: 1989
year: 1990
year: 1991
year: 1992
year: 1993
year: 1994
year: 1995
year: 1996
year: 1997
year: 1998
year: 1999
year: 2000
year: 2001
year: 2002
year: 2003
year: 2004
year: 2005
year: 2006
year: 2007
year: 1980
year: 1981
year: 1982
year: 1983
year: 1984
year: 1985
year: 1986
year: 1987
year: 1988
year: 1989
year: 1990
year: 1991
year: 1992
year: 1993
year: 1994
year: 1995
year: 1996
year: 1997
year: 1998
year: 1999
year: 2000
year: 2001
year: 2002
year: 2003
year: 2004
year: 2005
year: 2006
year: 2007
upper_year: 20
year: 1980
year: 1981
year: 1982
year: 1983
year: 1984
year: 1985
year: 1986
year: 1987
year: 1988
year: 1989
year: 1990
year: 1991
year: 1992
year: 1993
year: 1994
year: 1995
year: 1996
year: 1997
year: 1998
year: 1999
year: 2000
year: 2001
year: 2002
year: 1980
year: 1981
year: 1982
year: 1983
year: 1984
year: 1985
year: 1986
year: 1987
year: 1988
ye

In [15]:
intercept = {}
beta = {}
t_intercept = {}
t_beta = {}
r2 = {}

intercept_total = {}
beta_total = {}
t_intercept_total = {}
t_beta_total = {}
r2_total = {}

for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.Logit(depletion[upper_year], x_)#sm.OLS(depletion[upper_year], x_)
    res = mod.fit()
    print(res.summary())
#     print(res.prsquared)
    intercept[upper_year] = np.round(res.params[0],3)
    beta[upper_year] = np.round(res.params[1],3)
    
    t_intercept[upper_year] = np.round(res.tvalues[0],3)
    t_beta[upper_year] = np.round(res.tvalues[1],3)
    
    r2[upper_year] = np.round(res.prsquared, 3)
    
for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.OLS(wealth[upper_year], x_)
    res = mod.fit()
    print(res.summary())
#     print(res.prsquared)

    intercept_total[upper_year] = np.round(res.params[0],3)
    beta_total[upper_year] = np.round(res.params[1],3)
    
    t_intercept_total[upper_year] = np.round(res.tvalues[0],3)
    t_beta_total[upper_year] = np.round(res.tvalues[1],3)
    
    r2_total[upper_year] = np.round(res.rsquared_adj, 3)

upper_year: 15
         Current function value: 0.135830
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                   56
Model:                          Logit   Df Residuals:                       54
Method:                           MLE   Df Model:                            1
Date:                Sun, 09 Jul 2023   Pseudo R-squ.:                  0.1184
Time:                        22:38:54   Log-Likelihood:                -7.6065
converged:                      False   LL-Null:                       -8.6283
Covariance Type:            nonrobust   LLR p-value:                    0.1529
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             9.9401    122.838      0.081      0.936    -230.818     250.698
const        -12.7127    122.836     -0.103      0

/home/slcf/anaconda3/envs/hyungjinko/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/slcf/anaconda3/envs/hyungjinko/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


LinAlgError: Singular matrix

In [ ]:
#      0 / 5 / 10
# 8 -> 5.1 / 4.7 / 4.3
# 6 -> 3 / 2.6 / 2.1
# 4 -> 1.3 / 0.9 / 0.5

In [ ]:
print('Intercept & & {} & {} & & {} & {} & & {} & {}'.format(intercept[15], intercept_total[15], intercept[20], intercept_total[20], intercept[25], intercept_total[25]))
print(' & & ({}) & ({}) & & ({}) & ({}) & & ({}) & ({})'.format(t_intercept[15], t_intercept_total[15], t_intercept[20], t_intercept_total[20], t_intercept[25], t_intercept_total[25]))
print('Bearish (dummy) & & {} & {} & & {} & {} & & {} & {}'.format(beta[15], beta_total[15], beta[20], beta_total[20], beta[25], beta_total[25]))
print(' & & ({}) & ({}) & & ({}) & ({}) & & ({}) & ({})'.format(t_beta[15], t_beta_total[15], t_beta[20], t_beta_total[20], t_beta[25], t_beta_total[25]))



In [ ]:
print('Adj. $R^2$ & & {} & {} & & {} & {} & & {} & {}'.format(r2[15], r2_total[15], r2[20], r2_total[20], r2[25], r2_total[25]))


In [ ]:
upper_years = [10, 15, 20] #[15, 20, 25]#[10, 15, 20] # , 

# 10

In [ ]:
init_wealth = 350000
k = 0.5 # 고정

x = {
    10:[],
    15:[],
    20:[],
    25:[]
}
depletion = {
    10:[],
    15:[],
    20:[],
    25:[]
    
}
wealth = {
    10:[],
    15:[],
    20:[],
    25:[]
    
}

for upper_year in upper_years:
    print('upper_year:', upper_year)
    if upper_year == 10:
        withdraw_rates = [0.051, 0.03]
    elif upper_year == 15:
        withdraw_rates = [0.047, 0.026]
    elif upper_year == 20:
        withdraw_rates = [0.043, 0.021]
        
    for withdraw_rate in withdraw_rates:
        for years in full_year[upper_year]:
            for year in years:
                print('year:', year)
                if year in normal_year[upper_year][0]:
                    x[upper_year].append(0)
                elif year in bear_year[upper_year][0]:
                    x[upper_year].append(1)

                res = pv.get_metric(year, k, upper_year, withdraw_rate, df, init_wealth, p_val, w_val)
                if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                    dep_or_not = 1
                elif res[2] >= upper_year*12: # 고갈이 안되면
                    dep_or_not = 0

                depletion[upper_year].append(dep_or_not)
                wealth[upper_year].append((res[1] + res[0])*350000/100/1000)



In [ ]:
intercept = {}
beta = {}
t_intercept = {}
t_beta = {}
r2 = {}

intercept_total = {}
beta_total = {}
t_intercept_total = {}
t_beta_total = {}
r2_total = {}

for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.Logit(depletion[upper_year], x_)#sm.OLS(depletion[upper_year], x_)
    res = mod.fit()
    print(res.summary())
#     print(res.prsquared)
    intercept[upper_year] = np.round(res.params[0],3)
    beta[upper_year] = np.round(res.params[1],3)
    
    t_intercept[upper_year] = np.round(res.tvalues[0],3)
    t_beta[upper_year] = np.round(res.tvalues[1],3)
    
    r2[upper_year] = np.round(res.prsquared, 3)
    
for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.OLS(wealth[upper_year], x_)
    res = mod.fit()
    print(res.summary())
#     print(res.prsquared)

    intercept_total[upper_year] = np.round(res.params[0],3)
    beta_total[upper_year] = np.round(res.params[1],3)
    
    t_intercept_total[upper_year] = np.round(res.tvalues[0],3)
    t_beta_total[upper_year] = np.round(res.tvalues[1],3)
    
    r2_total[upper_year] = np.round(res.rsquared_adj, 3)

In [ ]:
print('Intercept & & {} & {} & & {} & {} & & {} & {}'.format(intercept[10], intercept_total[10], intercept[15], intercept_total[15], intercept[20], intercept_total[20]))
print(' & & ({}) & ({}) & & ({}) & ({}) & & ({}) & ({})'.format(t_intercept[10], t_intercept_total[10], t_intercept[15], t_intercept_total[15], t_intercept[20], t_intercept_total[20]))
print('Bearish (dummy) & & {} & {} & & {} & {} & & {} & {}'.format(beta[10], beta_total[10], beta[15], beta_total[15], beta[20], beta_total[20]))
print(' & & ({}) & ({}) & & ({}) & ({}) & & ({}) & ({})'.format(t_beta[10], t_beta_total[10], t_beta[15], t_beta_total[15], t_beta[20], t_beta_total[20]))



In [ ]:
print('Adj. $R^2$ & & {} & {} & & {} & {} & & {} & {}'.format(r2[10], r2_total[10], r2[15], r2_total[15], r2[20], r2_total[20]))
